# VNA V3

Want to be able to live plot both S-parameters and the data-read in at once. Also cleans up the imports.

In [ ]:
import sys
import os
import importlib #
sys.path.append(os.path.abspath('../scripts'))

#widgets and display
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import Output
import pprint

#Plotting 
from bqplot import pyplot as plt
import threading
from threading import Thread

#caculations
import numpy as np 
from scipy import signal
import time
import math

# Import functions from custom scripts
import sig_source, sig_source_ui, data_proccess, adl5960
importlib.reload(sig_source)
importlib.reload(sig_source_ui)
importlib.reload(data_proccess)
importlib.reload(adl5960)
from sig_source import SigSource
from sig_source_ui import SignalSourceUI
from data_proccess import VNAfunc
from adl5960 import ADL5960

In [ ]:
#Import Pynq
from pynq import PL, allocate, Overlay
import xrfdc

### Upload Code to RFSOC

In [ ]:
rfsoc_button = widgets.Button(description="Print RFSOC Code Lines")

rfosc_out = Output()
def print_rfsoc(func):
    with rfosc_out:
        rfosc_out.clear_output
        try: 
            pprint.pprint(ol.ip_dict)
        except Exception as e:
            print(f"Error: {e}")


PL.reset() #important fixes caching issues which have popped up.
ol = Overlay('../design_1.bit')  #locate/point to the bit file
dma_interface = ol.axi_dma_0
print(ol.ip_dict.keys())

rf = ol.usp_rf_data_converter_0

rfsoc_button.on_click(print_rfsoc)
display(widgets.VBox([widgets.Label(value="Print RFSOC Output"), rfsoc_button, rfosc_out])) #Display button only after the code is uploaded 

### Signal Source Generation 

In [ ]:
# Create instances and start for the LMX2595
adl1 = ADL5960(ol.spi_adl_0)  
adl2 = ADL5960(ol.spi_adl_1) 

# Create and display the UI for signal generation
sig_ui = SignalSourceUI(mmio_spi_controller = ol.spi_lmx_0)
sig_ui.display()

### Real Time Output Plot

In [ ]:
fs = 147.456e6 # Sampling frequency
n = 65536 # Number of samples
T = n/fs

In [ ]:
#Important Functions!
def read_dma():
    # Trigger the DMA transfer and wait for the result
    out_buffer = allocate(400024 * 4, dtype=np.int32)
    # Trigger the DMA transfer and wait for the result
    dma_interface.recvchannel.transfer(out_buffer)
    dma_interface.recvchannel.wait()
    return out_buffer

In [ ]:
'''
#Plot for Live-DMA Read

# Plot Size
PLOT_SIZE = 3e-6
# Initialize data lists
time_data = np.linspace(0, T, n)  # Time data (X-axis)
x_axis = [time_data, time_data, time_data, time_data]
# Initialize the plot
fig = plt.figure(title="Real-time Sensor Data", animation_duration=0)
line = plt.plot([], [], colors=["blue", "red", "green", "orange"])  # Initial empty plot
plt.xlim(0, PLOT_SIZE)  # Initial X-axis range 
plt.xlabel("Time [s]")

freq_list = sig_ui.source.generate_freq_points()
freq_list_length = len(freq_list)
#Setup lists for Sparam storage
plot_S11_mag = np.zeros(freq_list_length)
plot_S11_phase = np.zeros(freq_list_length)
plot_S12_mag = np.zeros(freq_list_length)
plot_S12_phase = np.zeros(freq_list_length)
plot_S21_mag = np.zeros(freq_list_length)
plot_S21_phase = np.zeros(freq_list_length)
plot_S22_mag = np.zeros(freq_list_length)
plot_S22_phase = np.zeros(freq_list_length)

# Function to update the plot with new sensor data
def update_plot():
    while True:
        if is_running.value:
            #Set frequency index here
            freq_index = sig_ui.source.get_current_index()

            #Read out DMA here
            out_buffer = read_dma()

            # Convert the entire out_buffer into a NumPy array
            out_buffer_np = np.array(out_buffer, dtype=np.int32)  # Assuming `out_buffer` is the raw input array

            # Split into 4 sub-arrays
            out_buffer0 = out_buffer_np[0::4]
            out_buffer1 = out_buffer_np[1::4]
            out_buffer2 = out_buffer_np[2::4]
            out_buffer3 = out_buffer_np[3::4]

            # Process all buffers at once
            real0, imag0 = VNAfunc.iq_break_data_np(out_buffer0)
            real1, imag1 = VNAfunc.iq_break_data_np(out_buffer1)
            real2, imag2 = VNAfunc.iq_break_data_np(out_buffer2)
            real3, imag3 = VNAfunc.iq_break_data_np(out_buffer3)
                
            #At some point here, need to insert a switch change function??

            #Caculate the S-parameters 
            #For caculating the S-parameters combine into complex lists
            array0 = real0 + 1j*imag0 
            array1 = real1 + 1j*imag1
            array2 = real2 + 1j*imag2 
            array3 = real3 + 1j*imag3 
            S11_mag, S11_phase = VNAfunc.caculate_S_param(array0, array1)
            S12_mag, S12_phase = VNAfunc.caculate_S_param(array0, array2)
            S21_mag, S21_phase = VNAfunc.caculate_S_param(array1, array3)
            S22_mag, S22_phase = VNAfunc.caculate_S_param(array2, array3)

            #Store the S-parameters into a different line
            # Store magnitude and phase for each S-parameter
            plot_S11_mag[freq_index] = S11_mag
            plot_S11_phase[freq_index] = S11_phase
            plot_S12_mag[freq_index] = S12_mag
            plot_S12_phase[freq_index] = S12_phase
            plot_S21_mag[freq_index] = S21_mag
            plot_S21_phase[freq_index] = S21_phase
            plot_S22_mag[freq_index] = S22_mag
            plot_S22_phase[freq_index] = S22_phase

            # Update the plot with new data
            line.x = x_axis
            line.y = [real0, real1, real2, real3]
            
# Toggle button to start/stop the plot
is_running = widgets.ToggleButton(
    value=False,
    description="Running",
    icon="play",
    tooltip="Start/Stop the live plot",
)

# Display the toggle button and plot
display(is_running, fig)

# Function to start the thread for continuous plotting
def start_plot(change):
    if is_running.value:
        # Run the update function in a separate thread to avoid blocking the main thread
        thread = Thread(target=update_plot, daemon=True)
        thread.start()

# Watch the button and start the plot when pressed
is_running.observe(start_plot, names='value')
'''

In [ ]:
'''
#For one pass of a frequency list 
frequency_list = sig_ui.source.generate_freq_points()

# Initialize empty arrays for S-parameters
plot_S11_mag = np.zeros(len(frequency_list))
plot_S11_phase = np.zeros(len(frequency_list))
plot_S12_mag = np.zeros(len(frequency_list))
plot_S12_phase = np.zeros(len(frequency_list))
plot_S21_mag = np.zeros(len(frequency_list))
plot_S21_phase = np.zeros(len(frequency_list))
plot_S22_mag = np.zeros(len(frequency_list))
plot_S22_phase = np.zeros(len(frequency_list))

#Iterate through frequencies and compute S-parameters
for i in range(1, len(frequency_list)): #skips the first set
    sig_ui.source.set_frequency(frequency_list[i])  # Set the current frequency
    out_buffer = read_dma()  # Read the data buffer
    S11_mag, S11_phase, S12_mag, S12_phase, S21_mag, S21_phase, S22_mag, S22_phase = VNAfunc.extract_S_param(out_buffer)
    
    # Store magnitude and phase for each S-parameter
    plot_S11_mag[i] = S11_mag
    plot_S11_phase[i] = S11_phase
    plot_S12_mag[i] = S12_mag
    plot_S12_phase[i] = S12_phase
    plot_S21_mag[i] = S21_mag
    plot_S21_phase[i] = S21_phase
    plot_S22_mag[i] = S22_mag
    plot_S22_phase[i] = S22_phase

'''

In [ ]:
#Integrated time and s-parameter domain plot 

# Constants
PLOT_SIZE = 3e-6  # Time plot size
freq_list = sig_ui.source.generate_freq_points()  # Frequency points from signal source
freq_list_length = len(freq_list)

# Initialize storage for S-parameters
plot_S11_mag = np.zeros(freq_list_length)
plot_S11_phase = np.zeros(freq_list_length)
plot_S12_mag = np.zeros(freq_list_length)
plot_S12_phase = np.zeros(freq_list_length)
plot_S21_mag = np.zeros(freq_list_length)
plot_S21_phase = np.zeros(freq_list_length)
plot_S22_mag = np.zeros(freq_list_length)
plot_S22_phase = np.zeros(freq_list_length)

# Shared data for time-domain plots
time_data = np.linspace(0, PLOT_SIZE, 1000)  # Example time data
real0, imag0, real1, imag1, real2, imag2, real3, imag3 = (
    np.zeros(1000) for _ in range(8)
)

# Time-domain Plot
fig_time = plt.figure(title="Real-time Sensor Data", animation_duration=0)
time_lines = plt.plot([], [], colors=["blue", "red", "green", "orange"])  # Time-domain lines
plt.xlim(0, PLOT_SIZE)
plt.xlabel("Time [s]")

# Frequency-domain Plot for Magnitude
fig_freq = plt.figure(title="S-Parameter Magnitude", animation_duration=0)
freq_lines = plt.plot(
    [], [],
    colors=["blue", "red", "green", "orange"],
    labels=["S11", "S12", "S21", "S22"]
)  # S-parameter magnitude lines
plt.xlim(freq_list[0], freq_list[-1])
plt.xlabel("Frequency [Hz]")
plt.legend()

# S-parameter Trace Toggles
sparam_toggles = {
    "S11": widgets.ToggleButton(value=True, description="S11", icon="eye"),
    "S12": widgets.ToggleButton(value=True, description="S12", icon="eye"),
    "S21": widgets.ToggleButton(value=True, description="S21", icon="eye"),
    "S22": widgets.ToggleButton(value=True, description="S22", icon="eye"),
}
toggle_box = widgets.HBox(list(sparam_toggles.values()))  # Horizontal box for toggle buttons

# Toggle button for live updates
is_running = widgets.ToggleButton(
    value=False,
    description="Running",
    icon="play",
    tooltip="Start/Stop the live plot",
)

# Function to update plots
def update_plots():
    global real0, imag0, real1, imag1, real2, imag2, real3, imag3
    while is_running.value:
        freq_index = sig_ui.source.get_current_index()

        # Read out DMA here
        out_buffer = read_dma()
        out_buffer_np = np.array(out_buffer, dtype=np.int32)

        # Split buffers and process data
        out_buffer0 = out_buffer_np[0::4]
        out_buffer1 = out_buffer_np[1::4]
        out_buffer2 = out_buffer_np[2::4]
        out_buffer3 = out_buffer_np[3::4]
        real0, imag0 = VNAfunc.iq_break_data_np(out_buffer0)
        real1, imag1 = VNAfunc.iq_break_data_np(out_buffer1)
        real2, imag2 = VNAfunc.iq_break_data_np(out_buffer2)
        real3, imag3 = VNAfunc.iq_break_data_np(out_buffer3)

        # Calculate S-parameters
        S11_mag, S11_phase = VNAfunc.caculate_S_param(real0 + 1j * imag0, real1 + 1j * imag1)
        S12_mag, S12_phase = VNAfunc.caculate_S_param(real0 + 1j * imag0, real2 + 1j * imag2)
        S21_mag, S21_phase = VNAfunc.caculate_S_param(real1 + 1j * imag1, real3 + 1j * imag3)
        S22_mag, S22_phase = VNAfunc.caculate_S_param(real2 + 1j * imag2, real3 + 1j * imag3)

        # Store S-parameter data
        plot_S11_mag[freq_index] = S11_mag
        plot_S12_mag[freq_index] = S12_mag
        plot_S21_mag[freq_index] = S21_mag
        plot_S22_mag[freq_index] = S22_mag

        # Update time-domain plot
        time_lines[0].x = time_data
        time_lines[0].y = real0
        time_lines[1].x = time_data
        time_lines[1].y = real1
        time_lines[2].x = time_data
        time_lines[2].y = real2
        time_lines[3].x = time_data
        time_lines[3].y = real3

        # Update frequency-domain plot with visibility toggles
        freq_lines[0].visible = sparam_toggles["S11"].value
        freq_lines[0].x = freq_list
        freq_lines[0].y = plot_S11_mag
        freq_lines[1].visible = sparam_toggles["S12"].value
        freq_lines[1].x = freq_list
        freq_lines[1].y = plot_S12_mag
        freq_lines[2].visible = sparam_toggles["S21"].value
        freq_lines[2].x = freq_list
        freq_lines[2].y = plot_S21_mag
        freq_lines[3].visible = sparam_toggles["S22"].value
        freq_lines[3].x = freq_list
        freq_lines[3].y = plot_S22_mag

        time.sleep(0.1)  # Delay for live update

# Function to start the update thread
def start_plots(change):
    if is_running.value:
        thread = Thread(target=update_plots, daemon=True)
        thread.start()

# Observe toggle state for start/stop
is_running.observe(start_plots, names="value")

# Display the UI components
display(widgets.VBox([toggle_box, is_running, widgets.HBox([fig_time, fig_freq])]))


NameError: name 'sig_ui' is not defined